mp4 to wav

In [1]:
import os
import glob
import ntpath
import sys

def createDirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

def convert_mp4_to_avi(file_name, output_directory):
    input_name = file_name
    output_name = ntpath.basename(file_name)
    output = os.path.join(output_directory, output_name.replace('.mp4', '.wav', 1))
    cmd = "ffmpeg -i {} -ab 160K -ac 1 -ar 16000 -vn {}".format(input_name, output) 
    return os.popen(cmd)

def main():
    input_directory = 'data/videos'
    output_directory = 'data/audios'
    createDirectory(output_directory)
    files = glob.glob(input_directory + '/*.mp4')
    for file_name in files:
        convert_mp4_to_avi(file_name, output_directory)
        print('convert mp4 to wav :', file_name)

if __name__ == "__main__":
   main()


In [1]:
%pwd
%cd ../..

/opt/ml/input/code/project


In [2]:
from torchvggish.torchvggish import vggish
import numpy as np
import glob

# 변환할 오디오 파일 위치 지정
audio_path = 'data/audios'
files = glob.glob(audio_path + '/*.wav')
file_name_list = os.listdir(audio_path)

# 오디오 피처 저장할 위치 지정
feature_save_path = 'data/audio_features'

# Initialise model and download weights
model_urls = {
    'vggish': 'https://github.com/harritaylor/torchvggish/'
    'releases/download/v0.1/vggish-10086976.pth',
    'pca': 'https://github.com/harritaylor/torchvggish/'
    'releases/download/v0.1/vggish_pca_params-970ea276.pth'
    }
embedding_model = vggish.VGGish(urls=model_urls, postprocess=False)
embedding_model.eval()

print("[saved list]")
saved_list = []
for audio, audio_name in zip(files, file_name_list):
    embeddings = embedding_model.forward(audio)
    np.save(os.path.join(feature_save_path, audio_name[:-4]),embeddings.detach().cpu().numpy())
    print(os.path.join(feature_save_path, audio_name[:-4])+".npy")
    saved_list.append(os.path.join(feature_save_path, audio_name[:-4])+".npy")

[saved list]
data/audio_features/Bad.Boys.1995__#01-33-51_01-34-37_label_B2-0-0.npy


In [3]:
for sample in saved_list:
    data = np.load(sample)
    print(f'{sample[len(feature_save_path)+1:]} - {data.shape}')

Bad.Boys.1995__#01-33-51_01-34-37_label_B2-0-0.npy - (69, 128)
